In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout, Box
from IPython.display import clear_output
import _pickle as pickle
from w2v_model import word2vec
import get_answer

In [5]:
ob = word2vec()
path = "../data_prep/data/"
res = get_answer.get_dir_list(path)
print (res)
get_answer.ob = ob
driver = get_answer.Driver()
fp = open(path+"./../PR.txt", 'w')
for cat in res:
    print ("\n\n\n\n==================================\n\n\n\n")
    print ("                 "+cat+"               ")
    print ("\n\n\n\n==================================\n\n\n\n")
    fp.write(cat)
    fp.write("\n")
    driver.run_w2v(path+cat, fp)
fp.close()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/shashwat/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/shashwat/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /home/shashwat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/shashwat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['Baby', 'Musical_Instruments', 'Office_Products', 'Patio_Lawn_and_Garden', 'PR.txt']








                 Baby               








loading data
reading dict
reading word vec
reading model
done








                 Musical_Instruments               








loading data
reading dict
reading word vec
reading model
done








                 Office_Products               








loading data
r

2018-04-23 01:38:40,770 : INFO : collecting all words and their counts
2018-04-23 01:38:40,771 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-23 01:38:40,805 : INFO : collected 89404 word types from a corpus of 89404 raw words and 1 sentences
2018-04-23 01:38:40,805 : INFO : Loading a fresh vocabulary


Performing Word2Vec


2018-04-23 01:38:41,179 : INFO : min_count=1 retains 89404 unique words (100% of original 89404, drops 0)
2018-04-23 01:38:41,179 : INFO : min_count=1 leaves 89404 word corpus (100% of original 89404, drops 0)
2018-04-23 01:38:41,346 : INFO : deleting the raw counts dictionary of 89404 items
2018-04-23 01:38:41,347 : INFO : sample=0.001 downsamples 0 most-common words
2018-04-23 01:38:41,348 : INFO : downsampling leaves estimated 89404 word corpus (100.0% of prior 89404)
2018-04-23 01:38:41,481 : INFO : estimated required memory for 89404 words and 100 dimensions: 116225200 bytes
2018-04-23 01:38:41,481 : INFO : resetting layer weights
2018-04-23 01:38:42,241 : INFO : training model with 3 workers on 89404 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2018-04-23 01:38:42,246 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-23 01:38:42,254 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-23 01:38:42,294 

Dumping Pickles
Total products with Questions = 1561
Processing








                 PR.txt               








creating data files
Reading files ../data_prep/data/PR.txt/PR.txt_Review.json


NotADirectoryError: [Errno 20] Not a directory: '../data_prep/data/PR.txt/PR.txt_Review.json'

In [4]:
result= None
def ask_question(b):
    global result
    global category, product, question_text    
    result = get_answer.review_2_sent(question_text.value, 4, product.value)

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn

stp = set(stopwords.words('english'))
word_type={}
word_type['JJ'] = 'a'
word_type['JJR'] = 'a'
word_type['JJS'] = 'a'
word_type['NN'] = 'n'
word_type['NNS'] = 'n'
word_type['NNP'] = 'n'
word_type['NNPS'] = 'n'
word_type['RB'] = 'r'
word_type['RBR'] = 'r'
word_type['RBS'] = 'r'
word_type['VB'] = 'v'
word_type['VBD'] = 'v'
word_type['VBG'] = 'v'
word_type['VBN'] = 'v'
word_type['VBP'] = 'v'
word_type['VBZ'] = 'v'

def remove_stopwords(words):
    newList = []
    for word in words:
        if word not in stp:
            newList.append(word)
    return newList
    
def binary_voting_function(text):
    token = nltk.word_tokenize(text)
    token = remove_stopwords(token)
    tagged = nltk.pos_tag(token)
    positive_score = 0.0
    negative_score = 0.0
    for each_tagged in tagged:
        term = each_tagged[0]
        tag_type = each_tagged[1]
        if tag_type in word_type:
            synset_list = wn.synsets(term)
            if len(synset_list)!=0:
                senti = synset_list[0].name()
                senti_obj = swn.senti_synset(senti)
                positive_score = positive_score + float(senti_obj.pos_score())
                negative_score = negative_score + float(senti_obj.neg_score())
    if (positive_score + negative_score == 0.0):
        return 0.0
    return float(positive_score) / float(positive_score + negative_score)


In [9]:
import json
import spacy
import wmd
import json
import os
from nltk.translate.bleu_score import sentence_bleu
import numpy as np
path = "../data_prep/data/"
res = get_answer.get_dir_list(path)
ob = word2vec()
print (res)
get_answer.ob = ob
final_results = {}
nlp = spacy.load('en', create_pipeline=wmd.WMD.create_spacy_pipeline)
for cat in res:
    print("starting for ", cat)
    folder = path+cat
    category = (folder.split('/'))[3]
    question_file = os.path.join(folder, category + "_QA.json")
    dict_file = os.path.join(folder, category + "_Dict.p")
    wordvec_file = os.path.join(folder, category + "_WordVec.p")
    model_file = os.path.join(folder, category + "_Model.p")
    ob.read_data(dict_file,wordvec_file,model_file)
    ob.init_word2vect(False)
    q_file = open(question_file)
    score = []
    score_wmd = []
    correctR = 0
    wrongR = 0
    for json_line in q_file:
        parsed_json = json.loads(json_line)
        print("parsing json for ", cat)
        count =0
        for line in parsed_json:
            p_id = line['asin']
            corr_ans = line['answer']
            ans_type = line['answerType']
            question = line['question']
            result = get_answer.review_2_sent(question,5,p_id)
            top_sentences = result['top5']
            pos =0
            for s in top_sentences[:2]:
                score.append(sentence_bleu(corr_ans, s[0]))
                pos+=binary_voting_function(s[0])
                res = nlp(s[0])
                temp = nlp(corr_ans)
                score_wmd.append(temp.similarity(res))
            if pos/5 >= 0.5:
                ans = 'Y'
            else:
                ans = 'N'
            if ans_type == 'Y' or ans_type == 'N':
                if ans_type == ans:
                    correctR+=1;
                else:
                    wrongR+=1;
            count+=1
            #print(count)
    result = {}
    result['correct'] = correctR
    result['wrong'] = wrongR
    result['BLEU'] = np.mean(score)
    result['WMD'] = np.mean(score_wmd)
    final_results[cat] = result
    print(cat)
    print("correct", final_results[cat]["correct"])
    print("wrong", final_results[cat]["wrong"])
    print("BLEU", final_results[cat]["BLEU"])
    print("WMD", final_results[cat]["WMD"])

print(final_results)
            

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/shashwat/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/shashwat/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /home/shashwat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/shashwat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['Musical_Instruments', 'PR.txt']
starting for  Musical_Instruments
reading dict
reading word vec
reading model
done
parsing json for  Musical_Instruments


/home/shashwat/anaconda2/envs/p3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/home/shashwat/anaconda2/envs/p3/lib/python3.6/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Musical_Instruments
correct 199
wrong 664
BLEU 0.683138330912101
WMD 0.5498961205784909
starting for  PR.txt


NotADirectoryError: [Errno 20] Not a directory: '../data_prep/data/PR.txt/PR.txt_Dict.p'